In [ ]:
pip install wildlife-datasets


In [ ]:
pip install wildlife-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

In [ ]:
pip install timm

**Importing Wildlife data set and Wildlife tools**



In [ ]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier

**MacaqueFaces Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET MacaqueFaces: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

**Importing the pretrain Model**

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query, database = extractor(dataset_query), extractor(dataset_database)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 49/49 [34:50<00:00, 42.66s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[0.93894947, 0.97903377, 0.9748248 , ..., 0.06067665, 0.0604975 ,
        0.07648014],
       [0.94975156, 0.9809722 , 0.9829178 , ..., 0.08857712, 0.08899386,
        0.10716249],
       [0.9368558 , 0.97414136, 0.9836564 , ..., 0.08185371, 0.08442868,
        0.10064524],
       ...,
       [0.9431046 , 0.985183  , 0.98194647, ..., 0.08535906, 0.08368647,
        0.10288547],
       [0.92794454, 0.98648065, 0.97138005, ..., 0.06948385, 0.06463583,
        0.08578961],
       [0.958699  , 0.9781159 , 0.9700406 , ..., 0.11579873, 0.11533174,
        0.13037327]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']
Accuracy on MacaqueFaces data: 100.00%


**LionData Dateset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET LionData: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.69s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 5/5 [03:30<00:00, 42.13s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[0.86896247, 0.69942033, 0.8777389 , ..., 0.829781  , 0.8954884 ,
        0.86451447],
       [0.9500992 , 0.7055931 , 0.9380013 , ..., 0.7477597 , 0.9478767 ,
        0.89982736],
       [0.87242615, 0.6950849 , 0.89943147, ..., 0.75892425, 0.91286826,
        0.9114601 ],
       ...,
       [0.8723602 , 0.7832219 , 0.9014683 , ..., 0.8094394 , 0.9129671 ,
        0.9049461 ],
       [0.8919123 , 0.73857903, 0.92439425, ..., 0.7845442 , 0.9392824 ,
        0.9575528 ],
       [0.89837366, 0.75758326, 0.91506803, ..., 0.80790067, 0.9342222 ,
        0.93150544]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on Lion data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['Sebastian' 'Saba' 'Etana' 'Tigisi' 'Sebastian' 'Moswen' 'Naape'
 'Simaloi' 'Autumn' 'Lolparpit' 'Maskio' 'Chaimu' 'Naape' 'Saba' 'Saitoti'
 'Esiriwua' 'Sadala' 'Etana' 'Enderoni' 'Kinna' 'Lucinda' 'Enkume' 'Joy'
 'Lemuanik' 'Naibor' 'Shambe' 'Siti' 'Selenkay' 'Sadala' 'Manzy' 'Moswen'
 'Napi' 'Ngare' 'Naimina' 'Napi' 'Cleopatra' 'Moswen' 'Kioni' 'Chaimu'
 'Engiyaa' 'Enderoni' 'Jamal' 'Leia' 'Sebastian' 'Nuru' 'Maskio' 'Mooza'
 'Moswen' 'Kibibi' 'Tigisi' 'Enderoni' 'Lucinda' 'Moswen' 'Kioni'
 'Sebastian' 'Mashavu' 'Mickey' 'Mama-Kali' 'Kioni' 'Ngare' 'Nashipai'
 'Sikio-Kali' 'Ngare' 'Amber' 'Olonyori' 'Lemuanik' 'Kibibi' 'Ngare'
 'Saimutie' 'Amber' 'Shemsa' 'Olonyori' 'Mickey' 'Esiriwua' 'Nashipai'
 'Enderoni' 'Summer' 'Nuru' 'Mickey' 'Joy' 'Cleopatra' 'Tigisi' 'Mooza'
 'Saitoti' 'Kioni' 'Naini' 'Rescue' 'Sikio-Kali' 'Empurra' 'Enadalut'
 'Lemayian' 'Rescue' 'Enadalut' 'Esiriwua' 'Enkume' 'Senteu' 'Doto'
 'Enkume' 'Nakato' 'Enderoni']
Accuracy on Lio

**NyalaDataSet**

In [ ]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET NyalaData: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 15/15 [10:46<00:00, 43.08s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[0.8619583 , 0.01217471, 0.05642469, ..., 0.6895717 , 0.03031028,
        0.898418  ],
       [0.25952083, 0.918558  , 0.785017  , ..., 0.5010886 , 0.82998675,
        0.1797384 ],
       [0.71705055, 0.4680481 , 0.5621629 , ..., 0.962554  , 0.5190879 ,
        0.7367308 ],
       ...,
       [0.2627344 , 0.9283303 , 0.7531287 , ..., 0.4816041 , 0.80183464,
        0.17906114],
       [0.84421116, 0.0514888 , 0.10151841, ..., 0.7681327 , 0.0811858 ,
        0.9342401 ],
       [0.07699895, 0.7878246 , 0.9208322 , ..., 0.38661018, 0.9427093 ,
        0.07355451]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['166' '10' '57' '89' '58' '179' '55' '58' '149' '71' '87' '125' '56'
 '144' '131' '8' '36' '150' '131' '11' '116' '19' '20' '86' '260' '165'
 '140' '11' '246' '41' '140' '197' '9' '131' '24' '48' '41' '11' '48'
 '131' '272' '47' '21' '46' '8' '241' '114' '62' '21' '48' '252' '151'
 '27' '108' '82' '51' '101' '85' '32' '58' '46' '139' '63' '85' '9' '181'
 '65' '242' '43' '144' '142' '12' '60' '5' '127' '166' '56' '21' '27' '36'
 '20' '188' '51' '51' '42' '20' '111' '62' '165' '68' '104' '174' '48'
 '217' '144' '6' '50' '91' '105' '140']
Accuracy on NyalaData data: 11.00%


**StripeSpotter Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET StripeSpotter: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.88s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 6/6 [04:25<00:00, 44.30s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[0.2130194 , 0.588757  , 0.65936303, ..., 0.4394258 , 0.44544154,
        0.621335  ],
       [0.17780262, 0.38624054, 0.40102234, ..., 0.7090034 , 0.20244715,
        0.2846234 ],
       [0.6676848 , 0.43850845, 0.34033626, ..., 0.2868997 , 0.39746365,
        0.36011013],
       ...,
       [0.60642976, 0.23701996, 0.21666232, ..., 0.41781956, 0.32831538,
        0.35214955],
       [0.51512223, 0.2798951 , 0.32729602, ..., 0.53868353, 0.39507398,
        0.28750026],
       [0.17924337, 0.24542159, 0.3013066 , ..., 0.58145046, 0.2895743 ,
        0.35858634]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['09_212' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '01_661'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP04_225' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']
Accuracy on Str

**IPanda50 Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=174954f6-d2bd-4289-b1ed-6c990eb70713
To: /data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:15<00:00, 59.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 3.27MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 21.5MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:38<00:00, 38.09s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 53/53 [38:16<00:00, 43.33s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[ 0.02714637,  0.53501487,  0.14056858, ...,  0.38829625,
         0.43481827, -0.07561179],
       [ 0.08373775,  0.5616951 ,  0.03785922, ...,  0.3602667 ,
         0.6493416 , -0.1295084 ],
       [ 0.01190435,  0.4141855 ,  0.23542283, ...,  0.45754877,
         0.5449476 , -0.1335805 ],
       ...,
       [ 0.0320384 ,  0.12872331, -0.14140168, ...,  0.02448   ,
         0.22118227, -0.17661369],
       [-0.08624166,  0.16572046,  0.07199848, ...,  0.14675027,
         0.23473434, -0.02312863],
       [ 0.01022829,  0.06892081,  0.7597883 , ...,  0.4224525 ,
         0.10611828, -0.04714176]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '37_xinger' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '38_xingfan' '04_chengdui' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '24_qixi' '10_hexing' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '10_hexing'
 '29_shuangxiong' '11_jiaoao' '44_yayi' '14_maodou' '30_shuqing'
 '36_xingda' '39_xinghui' '33_wuyi' '10_hexing' '10_hexing' '00_aibang'
 '49_yuanrun' '35_xilan' '04_chengdui' '10_hexing' '39_xinghui' '44_yayi'
 '10_hexing' '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui'
 '10_hexing' '17_meibang' '00_aibang' '46_yazhu' '14_maodou' '15_maosun'
 '32_susu' '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomia

CZoo Dataset

In [ ]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

DATASET CZoo: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:35<00:00, 35.28s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 16/16 [10:51<00:00, 40.69s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[ 9.0549982e-01,  8.9250720e-01,  8.9013600e-01, ...,
        -2.4628341e-02, -1.9136328e-02,  2.0032311e-02],
       [ 9.1913915e-01,  9.1288275e-01,  9.0220594e-01, ...,
        -1.2885875e-02, -3.0940475e-02, -7.5778235e-03],
       [ 9.3326151e-01,  9.2265338e-01,  9.1522133e-01, ...,
        -3.5657752e-02, -1.6588740e-02,  1.8029639e-02],
       ...,
       [ 9.4986129e-01,  9.6489835e-01,  9.4513679e-01, ...,
        -5.2607138e-02, -3.3724420e-03, -1.1571882e-02],
       [ 9.5598102e-01,  9.6259630e-01,  9.4699091e-01, ...,
        -4.5473136e-02, -1.4542699e-02, -3.2175947e-03],
       [ 9.4659024e-01,  9.6087039e-01,  9.4981194e-01, ...,
        -4.7210053e-02, -3.1141117e-03, -8.7950751e-04]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision
precision = precision_score(dataset_query.labels_string, predictions, average='weighted')

# Calculate recall
recall = recall_score(dataset_query.labels_string, predictions, average='weighted')

# Calculate F1 score
f1 = f1_score(dataset_query.labels_string, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 1.0
Recall: 1.0
F1 Score: 1.0
